# Model

The purpose of this script is to model churn based on engineered features
split the transformed feature data into a training and test set

Run this script after feature.ipynb

Requires features.p file from features.ipynb
Produces results.p and results.tsv

Features are 

Scaling is performed using MinMaxScaler

Categorical variables (e.g. tenant_id are encoded using OneHotEncoder)

Train_test_split was used to split the data into a training set and testing set

Several modeling approaches are tried including \
LogisticRegression, LinearSVC, KNeighborsClassifier, RandomForestClassifier

Cross-validation was performed (usually using built-in settings of the modeling object)

Evaluation was done using confusion matrix, ROC curve (and AUC metric)



# Load Libraries

In [ ]:
#Import libraries
%matplotlib inline
import logging
import collections
import datetime as dt
import sys
import os
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.figure 
import seaborn as sns

from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.svm import *
from sklearn.neighbors import *
from sklearn.ensemble import *
from sklearn.metrics import *
from sklearn.pipeline import Pipeline

# Load data

Note data cleaning was done in previous scripts.

In [ ]:
#Load features
# dfFeatures = pickle.load( open( "features_all.p", "rb" ) )
dfFeatures = pickle.load( open( "combined_features.p", "rb" ) )
dfFeatures.head()

In [ ]:
list(dfFeatures.columns)

In [ ]:
# Here's an opportunity to subset for particular groups of interest
#dfFeatures = dfFeatures[dfFeatures.period_count==1] # clients who have only had one subscription
#dfFeatures = dfFeatures[dfFeatures.num_periods==1] # clients who have only had one subscription

# Data cleaning (double check)

In [ ]:
# Check for empty values
print dfFeatures.info()
dfFeatures = dfFeatures.dropna()
print dfFeatures.info()

In [ ]:
dfFeatures.info()

In [ ]:
dfFeatures.describe()

In [ ]:
dfFeatures.head()

In [ ]:
#dfFeatures.ix[3,:]

# Assess Class Balance

In [ ]:
# Assess balance between classes
#dfFeatures.churned.unique()
churnedCount = np.sum(dfFeatures.churned==1) # Class 1 churn
engagedCount = np.sum(dfFeatures.churned==0) # Class 0 not churn
print churnedCount
print engagedCount
print str(round((float(churnedCount)/float(churnedCount+engagedCount))*100,1)) + '%'

# Scale data

In [ ]:
# Scale data
scaler = MinMaxScaler() # Also tried StandardScaler()
# without categoricals
masterList = [
#  'tenant_id',
 'num_days',
#  'num_interactions',
#  'num_emails',
#  'num_calls',
#  'num_meetings',
 'mean_gap',
#  'max_gap',
#  'min_gap',
#  'frequency',
 'email_frequency',
 'call_frequency',
 'meeting_frequency',
#  'num_periods',
#  'mean_duration',
#  'total_duration',
#  'min_duration',
#  'max_duration',
 'recency',
#  'churned'
]

# with categoricals
masterListPlus = [
 'tenant_id',
 'client_id',
 'num_days',
#  'num_interactions',
#  'num_emails',
#  'num_calls',
#  'num_meetings',
 'mean_gap',
#  'max_gap',
#  'min_gap',
#  'frequency',
 'email_frequency',
 'call_frequency',
 'meeting_frequency',
#  'num_periods',
#  'mean_duration',
#  'total_duration',
#  'min_duration',
#  'max_duration',
 'recency',
 'churned'
]

features = dfFeatures[masterList].as_matrix()
features_scaled = scaler.fit_transform(features)
dfFeaturesScaled = pd.DataFrame(features_scaled,columns=masterList)
dfFeaturesScaled['tenant_id'] = dfFeatures['tenant_id']
dfFeaturesScaled['client_id'] = dfFeatures['client_id']
dfFeaturesScaled['churned'] = dfFeatures['churned']
dfFeaturesScaled = dfFeaturesScaled[masterListPlus]
dfFeaturesScaled = dfFeaturesScaled.dropna()
featuresScaled = dfFeaturesScaled.as_matrix()

In [ ]:
# Define feature list
ylist = [
    'client_id',
    'churned'
]

#Different variations possible for features used
#... agg, first & last
#xlist=['email','call','meeting','email_first','call_first','meeting_first','email_last','call_last','meeting_last','avg_interval','period_duration_sum','period_count']

# agg & last
#xlist=['email','call','meeting','email_last','call_last','meeting_last','avg_interval','period_duration_sum','period_count']

# just last
#xlist=['email_last','call_last','meeting_last','avg_interval','period_duration_sum','period_count']

# just agg
#xlist=['email','call','meeting','avg_interval','days_since_last_touch','period_count','period_duration_sum']

# just agg w no period duration sum
#xlist=['email','call','meeting','avg_interval','days_since_last_touch','period_duration_sum']

# just agg
#xlist=['email','call','meeting','avg_interval','period_duration_sum','period_duration_mean','period_count']

#xlist=['email','call','meeting','email_first','call_first','meeting_first','email_last','call_last','meeting_last','avg_interval','period_duration_sum','period_count','days_since_last_touch']

xlist=[
 'tenant_id',
#  'client_id',
 'num_days',
#  'num_interactions',
#  'num_emails',
#  'num_calls',
#  'num_meetings',
 'mean_gap',
#  'max_gap',
#  'min_gap',
#  'frequency',
 'email_frequency',
 'call_frequency',
 'meeting_frequency',
#  'num_periods',
#  'mean_duration',
#  'total_duration',
#  'min_duration',
#  'max_duration',
#  'active_count',
 'recency',
#  'churned'
]

Y_lbl = dfFeaturesScaled['client_id']
dfY = dfFeaturesScaled['churned']
y = dfFeaturesScaled['churned'].as_matrix()

X_lbl = dfFeaturesScaled['tenant_id']
dfX = dfFeaturesScaled[xlist]
X = dfX.as_matrix()

In [ ]:
dfX.head()
dfX.describe()
dfX.info()
dfY.head()
X
y

# Create train and test sets

In [ ]:
#Split data into test set and training set
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [ ]:
#Show contents of raw data, training set and test set
print 'X'
print X
print
print 'y'
print y
print
print 'X_train'
print X_train
print
print 'X_test'
print X_test
print
print 'y_train'
print y_train
print
print 'y_test'
print y_test

In [ ]:
#Check sizes of raw data, training set and test set
print 'X'
print X.shape
print 'y'
print y.shape
print 'X_train'
print X_train.shape
print 'X_test'
print X_test.shape
print 'y_train'
print y_train.shape
print 'y_test'
print y_test.shape

In [ ]:
dfX = pd.DataFrame(X,columns=xlist)
dfY = pd.DataFrame(y,columns=['churned',])
dfXTrain = pd.DataFrame(X_train,columns=xlist)
dfYTrain = pd.DataFrame(y_train,columns=['churned',])
dfXTest = pd.DataFrame(X_test,columns=xlist)
dfYTest = pd.DataFrame(y_test,columns=['churned',])

In [ ]:
dfX = dfX[xlist][1:-1].apply(pd.to_numeric)
dfXTrain = dfXTrain[xlist][1:-1].apply(pd.to_numeric)
dfXTest = dfXTest[xlist][1:-1].apply(pd.to_numeric)
dfX['tenant_id'] = dfX['tenant_id'].astype(str)
dfXTrain['tenant_id'] = dfXTrain['tenant_id'].astype(str)
dfXTest['tenant_id'] = dfXTest['tenant_id'].astype(str)

In [ ]:
# print dfX.head()
# print
# print dfY.head()
# print
# print dfXTrain.head()
# print
# print dfXTest.head()
# print
# print dfYTrain.head()
# print
# print dfYTest.head()
# print

In [ ]:
# dfX.info()
# print
# dfY.info()
# print
# dfXTrain.info()
# print
# dfYTrain.info()
# print
# dfXTest.info()
# print
# dfYTest.info()
# print

# Feature Exploration

In [ ]:
#Feature correlation
ax = sns.heatmap(dfXTrain.corr())

In [ ]:
# Chi-squared statistic on feature importance
scores, pvalues = chi2(X_train, y_train)
print scores
print pvalues

In [ ]:
# Recursive feature elimination
clf = LogisticRegressionCV(cv=5,class_weight='balanced')
rfe = RFE(clf, 1)
rfe.fit(X_train,y_train)
#print range(0,len(xlist))
#print xlist
#print rfe.support_
#print rfe.ranking_

In [ ]:
dfFeatureSelection = pd.DataFrame({'feature':xlist,'score':scores,'p-value':pvalues,'rank':rfe.ranking_})

In [ ]:
dfFeatureSelectionSig = dfFeatureSelection[dfFeatureSelection['p-value']>0.05]
dfFeatureSelectionSig

In [ ]:
dfFeatureSelectionRank = dfFeatureSelection.sort_values('rank',ascending=True)
dfFeatureSelectionRank

In [ ]:
dfFeatureSelectionCombo = dfFeatureSelection[dfFeatureSelection['p-value']>0.05]
dfFeatureSelectionCombo = dfFeatureSelectionCombo.sort_values('rank',ascending=True)
dfFeatureSelectionCombo

In [ ]:
#Check sizes of raw data, training set and test set
print 'X'
print X.shape
print 'X_train'
print X_train.shape
print 'X_test'
print X_test.shape
print 'y'
print y.shape
print 'y_train'
print y_train.shape
print 'y_test'
print y_test.shape

In [ ]:
xlistReduced=[
 'tenant_id',
 'num_days',
#  'num_interactions',
 'num_emails',
 'num_calls',
 'num_meetings',
 'mean_gap',
#  'max_gap',
#  'min_gap',
#  'frequency',
 'email_frequency',
 'call_frequency',
 'meeting_frequency',
 'num_periods',
#  'mean_duration',
#  'total_duration',
#  'min_duration',
#  'max_duration'
 'recency'
]
# dfX = dfX[xlistReduced]
# dfXTrain = dfXTrain[xlistReduced]
# dfXTest = dfXTest[xlistReduced]

# X = dfX.as_matrix()
# X_train = dfXTrain.as_matrix()
# X_test = dfXTest.as_matrix()

In [ ]:
#Check sizes of raw data, training set and test set
print 'X'
print X.shape
print 'X_train'
print X_train.shape
print 'X_test'
print X_test.shape
print 'y'
print y.shape
print 'y_train'
print y_train.shape
print 'y_test'
print y_test.shape

# Encode Categoricals

In [ ]:
# Encode categoricals
enc = OneHotEncoder(categorical_features=[0,])
X_enc = enc.fit(X)
X_train_enc = enc.transform(X_train)
X_test_enc = enc.transform(X_test)

# Train a model

In [ ]:
# Several options make sense for modeling, including logistic regression, linear support vector classifier, 
# K-nearest neighbors, and random forest

clf = LogisticRegressionCV(cv=5,class_weight='balanced') #,penalty='l1',solver='liblinear')
#clf = LinearSVC(class_weight='balanced')
#clf = KNeighborsClassifier() #NO CLASS WEIGHT
#clf = RandomForestClassifier(class_weight='balanced') # n_estimators=15,


# Pipeline methods can be used to bundle feature selection process and classification
# clf = Pipeline([
#   ('feature_selection', SelectFromModel(linear_model.LogisticRegressionCV(cv=5,class_weight='balanced',penalty='l2'))),
#   ('classification', svm.LinearSVC(class_weight='balanced'))
# ])

# clf = Pipeline([
#   ('feature_selection', SelectFromModel(linear_model.LogisticRegressionCV(cv=5,class_weight='balanced',penalty='l2'))),
#   ('classification', ensemble.RandomForestClassifier(n_estimators=15,class_weight='balanced'))
# ])

def train_and_evaluate(clf, X_train, y_train):
    clf.fit(X_train,y_train)
    #sample weight available for LogisticRegressionCV, linearSVC, RandomForestClassifier
    #cv = KFold(n_splits=5,shuffle=True,random_state=None)
    scores = cross_val_score(clf, X_train, y_train) #,cv=cv)

    return clf

#scores

In [ ]:
# Train model
train_and_evaluate(clf,X_train_enc,y_train) #X_train,y_train)

# Evaluate Performance on a Test Set

In [ ]:
# Testing and evaluation # including ROC curve

global y_decision_function
global y_predict_proba
global y_predict
global y_score
def TestAndEvaluate(X_test, y_test, clf):
    try:
        y_decision_function = clf.decision_function(X_test) #Regression, linearSVC
        print 'y_decision_function'
        print y_decision_function  
        
        y_ret = y_decision_function
    except Exception as err:
        logging.exception(err)
    
    try:
        y_predict_proba = clf.predict_proba(X_test)[:,1] #KNeighborsSVC, Random forest
        print 'y_predict_proba'
        print y_predict_proba
        
        y_ret = y_predict_proba
    except Exception as err:
        logging.exception(err)

    try:
        y_predict = clf.predict(X_test) #[:,1]
        print 'y_predict'
        print y_predict
        
        y_ret = y_predict
    except Exception as err:
        logging.exception(err)

    try:
        y_score = clf.score(X_test,y_test) #[:,1]
        print 'y_score'
        print y_score
        
        y_ret = y_score
    except Exception as err:
        logging.exception(err)

    try:
        fpr, tpr, _ = roc_curve(y_test, y_decision_function)
    except Exception as err:
        logging.exception(err)

    try:
        fpr, tpr, _ = roc_curve(y_test, y_predict_proba)
    except Exception as err:
        logging.exception(err)

    try:
        roc_auc = auc(fpr,tpr)   
        plt.figure()
        lw = 2
        plt.plot(fpr, tpr, color='darkorange',lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic')
        plt.legend(loc="lower right")
        plt.savefig('auc.png')
        plt.close()
    except Exception as err:
        logging.exception(err)

    try:
        features = SelectFromModel(clf,prefit=True)
        print features
    except Exception as err:
        logging.exception(err)
        
    try: # Attribute not available for all types of classifiers
        print 'Feature importances'
        print clf.feature_importances_
    except Exception as err:
        logging.exception(err)

    try: # Attribute not available for all types of classifiers
        print 'coef'
        print clf.coef_
    except Exception as err:
        logging.exception(err)
    
    return y_predict

In [ ]:
# Confusion matrix

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    print 'plot_confusion_matrix'
    cm = cm.astype('float') / cm.sum(axis=1)[:,np.newaxis]
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(2)
    classes=['engaged','churned']
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks)
    plt.yticks(tick_marks)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid(False)
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.savefig('cm.png')
    plt.close()
    return

def matrix_and_pars(ytest,ypred):
    print 'matrix_and_pars'
    cm = confusion_matrix(ytest,ypred)
    precision = float(cm[0][0]) / (cm[0][0]+cm[1][0])
    recall = float(cm[0][0]) / (cm[0][0]+cm[0][1])
    F1 = 2*precision*recall/(recall+precision)
    print 'recall: %0.3f precision: %0.3f F1: %0.3f' %(recall,precision,F1)
    print '%d %d' %(cm[0][0],cm[0][1])
    print '%d %d' %(cm[1][0],cm[1][1])
    plot_confusion_matrix(cm)
    return

In [ ]:
#y_pred = measure_performance(X_test_enc, y_test, clf)
y_pred = TestAndEvaluate(X_test_enc, y_test, clf)


In [ ]:
# Generate confusion matrix
matrix_and_pars(y_test,y_pred)

In [ ]:
# Feature importances plot
def FeatureImportances(clf):
    importances = list(clf.coef_[0,:xlen])
    importances = [abs(number) for number in importances]
    xlbls=range(xlen)
    print xlbls #numbers
    print xlist #names
    print importances
    plt.barh(xlbls,importances,tick_label=xlist, align='center')

In [ ]:
xlist = [x.replace('tenant_id', 'tenant') for x in xlist]
xlist = [x.replace('num_days', 'relationship length') for x in xlist]
xlist = [x.replace('email_frequency', 'email frequency') for x in xlist]
xlist = [x.replace('call_frequency', 'call frequency') for x in xlist]
xlist = [x.replace('meeting_frequency', 'meeting frequency') for x in xlist]
xlist = [x.replace('mean_gap', 'gap (avg)') for x in xlist]
xlist = [x.replace('max_gap', 'gap (max)') for x in xlist]
xlist = [x.replace('mean_duration', 'subscription duration') for x in xlist]
xlen = len(xlist)
FeatureImportances(clf)

# Output of data for dashboard

In [ ]:
#Run on all data (for dashboard)
X_all=enc.transform(X)
y_predict = clf.predict(X_all)
y_predict_proba = clf.predict_proba(X_all)
dfYLbl = pd.DataFrame(Y_lbl).reset_index()
dfY = pd.DataFrame(dfY).reset_index()
dfX = dfX.reset_index()
dfYPredict = pd.DataFrame(y_predict)
dfYPredict = dfYPredict.rename(columns={0:'churn_pred'})
dfYPredictProba = pd.DataFrame(y_predict_proba)
dfYPredictProba = dfYPredictProba.rename(columns={0:'churn_no'})
dfYPredictProba = dfYPredictProba.rename(columns={1:'churn_yes'})

In [ ]:
dfResult = dfYPredictProba.join(dfYPredict)
dfResult = dfFeaturesScaled.join(dfResult)
dfResult.head()

In [ ]:
dfResult.info()

In [ ]:
dfResult.describe()

In [ ]:
pickle.dump(dfResult, open( "results.p", "wb" ))
dfResult.to_csv('results.tsv',sep='\t')

In [ ]:
dfResult = dfYPredictProba.join(dfYPredict)
dfResult = dfFeatures.join(dfResult)
dfResult.head()

In [ ]:
dfResult.info()

In [ ]:
dfResult.describe()

In [ ]:
pickle.dump(dfResult, open( "results_int.p", "wb" ))
dfResult.to_csv('results_int.tsv',sep='\t')

# Conclusion

The likelihood of churn is driven by the total duration of the relationship more than other factors and, while predictions based on this feature are reliable, cadence of communication remains challenging area for exploration.